In [ ]:
import os, sys
import requests
import json
import datetime
import math
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    print("OPENAI_API_KEY not loaded.")
    sys.exit(1)

openai = OpenAI()

In [ ]:
X_RAPID_API_KEY = os.getenv('X_RAPID_API_KEY')
X_RAPID_API_HOST = os.getenv('X_RAPID_API_HOST')

DEMO = False
if not (X_RAPID_API_KEY or X_RAPID_API_HOST):
    print("X_RAPID_API_KEY OR HOST not loaded.")
    DEMO = True

if DEMO:
    print("Running in demo mode.")

In [ ]:
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

def epoch_to_datetime(epoch):
    dt_object = datetime.datetime.fromtimestamp(epoch)
    return dt_object.strftime("%d/%m/%Y %H:%M:%S")

In [ ]:
def get_weather(lat: float, lon: float, demo: bool = True):
    if demo:
        with open('weather/irakleio.json') as f:
            file_content = f.read()
        json_content = json.loads(file_content)
    else:
        url = "https://open-weather13.p.rapidapi.com/fivedaysforcast"
        headers = {
        	"x-rapidapi-key": X_RAPID_API_KEY,
        	"x-rapidapi-host": X_RAPID_API_HOST
        }
        params = {
            "latitude": lat,
            "longitude": lon,
            "lang": "EN"
        }
        response = requests.get(url, headers=headers, params=params)
        json_content = response.json()

    # Weather fields
    today = json_content.get("list", "")[0].get("main", "")
    temp = math.ceil(kelvin_to_celsius(today.get("temp", "")))
    feels_like = math.ceil(kelvin_to_celsius(today.get("feels_like", "")))
    temp_min = math.ceil(kelvin_to_celsius(today.get("temp_min", "")))
    temp_max = math.ceil(kelvin_to_celsius(today.get("temp_max", "")))
    pressure = today.get("pressure", "")
    sea_level = today.get("sea_level", "")
    humidity = today.get("humidity", "")
    # City fields
    city = json_content.get("city", "")
    city_name = city.get("name", "")
    city_coord = city.get("coord", "")
    city_country = city.get("country", "")
    city_population = city.get("population", "")
    city_sunrise = epoch_to_datetime(city.get("sunrise", ""))
    city_sunset = epoch_to_datetime(city.get("sunset", ""))
    # Return
    message = f"Today's weather in {city_name} - {city_country} (population: {city_population}) is {temp}°C (feels like: {feels_like}°C) with a minimum of {temp_min}°C and a maximum of {temp_max}°C. "
    message += f"The humidity is {humidity}, the pressure is {pressure} and the sea level is {sea_level}. "
    message += f"The sunrise is at {city_sunrise} and the sunset is at {city_sunset}."
    return message

In [ ]:
weather_function = {
    "name": "get_weather",
    "description": "Get the weather given a latitude and longitude. Call this whenever you need to know the weather for a specific location (latitude, longitude), for example when a customer asks 'What is the weather like in 35.3220497, 25.1144712?'",
    "parameters": {
        "type": "object",
        "properties": {
            "lat": {
                "type": "number",
                "description": "The latitude of the location"
            },
            "lon": {
                "type": "number",
                "description": "The longitude of the location"
            },
            "demo": {
                "type": "boolean",
                "description": "A boolean demo value to fetch data from the API or from the demo values"
            }
        },
        "required": ["lat", "lon"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [
    {
        "type": "function",
        "function": weather_function
    }
]

In [ ]:
def stream_gpt(prompt):
    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a helpful weather assistant"},
            {"role": "user", "content": prompt}
        ],
        stream=True,
        tools=tools
    )

    result = ""
    tool_call_response = None
    collected_chunks = []
    tool_calls = []
    for chunk in stream:
        delta = chunk.choices[0].delta
        collected_chunks.append(chunk)

        if hasattr(delta, "content") and delta.content:
            result += delta.content or ""
            yield result

        elif hasattr(delta, "tool_calls") and delta.tool_calls:
            for call in delta.tool_calls:
                tool_calls.append(call)

    if tool_calls:
        full_args = ""
        tool_name = tool_calls[0].function.name
        for call in tool_calls:
            full_args += call.function.arguments

        print(f"Tool call triggered: {tool_name} with args {full_args}")

        try:
            args = json.loads(full_args)
            if tool_name == "get_weather":
                weather = get_weather(args.get("lat", ""), args.get("lon", ""), DEMO)
                yield weather
        except Exception as e:
            yield f"\nError processing tool call: {str(e)}"

In [ ]:
view = gr.Interface(
    fn = stream_gpt,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6),
    ],
    outputs = [
        gr.Markdown(label = "Reponse")
    ],
    flagging_mode = "never",
    css = "footer{display: none !important}"
)
view.launch()